## **Data Cleaning**

The raw dataset underwent several preprocessing steps, including the removal of duplicates, handling of null values, type conversions, and the extraction of structured information from text fields (e.g., date and address columns) to ensure accuracy and consistency in the analysis.

---

### **Importing Necessary Libraries**

To initiate the data analysis, only the necessary Python library was imported. In this case, **pandas** was used for efficient data manipulation and analysis.

In [43]:
import pandas as pd

### **Pulling Raw Data**

The raw sales data was imported into the workspace. This dataset contains transactional records including order details, product information, pricing, and customer purchase addresses.

In [44]:
df = pd.read_csv('../data/raw_data.csv')

### **Initial Inspection of the Dataset**

To understand the structure and contents of the dataset, the first five rows were displayed using the `.head()` method. This provides a quick overview of the columns, data types, and potential quality issues such as missing or inconsistent values.

In [45]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


### **Column Name Standardization**

To ensure consistency and avoid potential issues during analysis, all column names were standardized. This includes:

- Removing leading/trailing whitespaces
- Converting all names to lowercase
- Replacing spaces with underscores

In [46]:
# Formatting column names

df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

### **Checking and Dropping Completely Null Rows**

To identify and remove rows that contain only missing values across all columns, the following check was performed:

In [47]:
# Check for null rows
df[df.isna().all(axis=1)].shape[0]

545

In [48]:
df.dropna(how='all', inplace=True)

### **Missing Values Per Column**

To assess the completeness of the dataset, the number of missing values in each column was calculated:

In [49]:
df.isna().sum()

order_id            0
product             0
quantity_ordered    0
price_each          0
order_date          0
purchase_address    0
dtype: int64

### **Checking for Duplicate Rows**

Since there are no missing values in any column, the next step is to identify any duplicate entries in the dataset. Duplicate rows can skew analysis, especially in aggregations and trends.

In [50]:
# Check for duplicates

df.duplicated().sum()

np.int64(618)

In [51]:
# Inspecting duplicates

df[df.duplicated(keep=False)]

,order_id,product,quantity_ordered,price_each,order_date,purchase_address
30,176585,Bose SoundSport Headphones,1,99.99,04/07/19 11:31,"823 Highland St, Boston, MA 02215"
31,176585,Bose SoundSport Headphones,1,99.99,04/07/19 11:31,"823 Highland St, Boston, MA 02215"
519,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1149,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1155,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
...,...,...,...,...,...,...
186738,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
186781,259296,Apple Airpods Headphones,1,150,09/28/19 16:48,"894 6th St, Dallas, TX 75001"
186782,259296,Apple Airpods Headphones,1,150,09/28/19 16:48,"894 6th St, Dallas, TX 75001"
186784,259297,Lightning Charging Cable,1,14.95,09/15/19 18:54,"138 Main St, Boston, MA 02215"


### **Removing Duplicate Rows**

After identifying the number of duplicate rows, they were removed to ensure data accuracy and integrity using the following command:

In [52]:
df.drop_duplicates(inplace=True)

### **Inspecting Column Data Types**

To understand the structure of the dataset, the data types of each column were checked using:

In [53]:
# Checking data types

df.dtypes

order_id            object
product             object
quantity_ordered    object
price_each          object
order_date          object
purchase_address    object
dtype: object

### **Converting Column Data Types**

Several columns were found to have incorrect data types that could affect calculations and time-based analysis:

- `order_id`, `quantity_ordered` were stored as object types instead of integers.
- `price_each` was stored as an object instead of float.
- `order_date` was stored as an object instead of `datetime`.

To fix this, the following conversions were applied:

In [54]:
# Supposed to be converting to right data types

df['order_id'] = df['order_id'].astype(int)
df['quantity_ordered'] = df['quantity_ordered'].astype(int)
df['price_each'] = df['price_each'].astype(float)
df['order_date'] = pd.to_datetime(df['order_date'], format='%Y-%m-%d %H:%M:%S')

ValueError: invalid literal for int() with base 10: 'Order ID'

### **Inspecting Data Type Conversion Error**

During the data type conversion process, an error was encountered in the `order_id` column. Specifically, the column contained an object value `'Order ID'`, which is not compatible with integer conversion.

This issue typically arises from duplicated header rows within the dataset. To investigate the cause, the following command was used:

In [55]:
df[df['order_id'] == 'Order ID']

,order_id,product,quantity_ordered,price_each,order_date,purchase_address
519,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


### **Dropping Invalid Header Row**

The inspection revealed that the row containing `'Order ID'` in the `order_id` column was a duplicated header row with no valid data.
Since this row does not contain any necessary or usable data, it was removed from the dataset using the following command:

In [56]:
# Dropping rows with 'Order ID' value in order_id column

df.drop(df[df['order_id'] == 'Order ID'].index, inplace=True)

### **Converting Data Types**

After cleaning the data and removing invalid entries, the necessary columns were converted to appropriate data types to enable accurate analysis.

In [57]:
# Converting Data types

df['order_id'] = df['order_id'].astype(int)
df['quantity_ordered'] = df['quantity_ordered'].astype(int)
df['price_each'] = df['price_each'].astype(float)
df['order_date'] = pd.to_datetime(df['order_date'], format='%m/%d/%y %H:%M')
df['purchase_address'] = df['purchase_address'].astype(str)

In [58]:
df.dtypes

order_id                     int64
product                     object
quantity_ordered             int64
price_each                 float64
order_date          datetime64[ns]
purchase_address            object
dtype: object

### **Splitting Address into Components**

The `purchase_address` column was split into three parts to extract key location details. This separation provided individual columns for the street, city, and a combined state and ZIP code. Extracting these components helps enable deeper geographic analysis in later stages of the project.

In [59]:
# extracting street and city
address_split = df['purchase_address'].str.split(',',expand=True)

###  **Inspecting `purchase_address` Format**

A sample of the `purchase_address` column was reviewed to understand its structure and confirm consistency. This inspection served as a reference for accurately extracting the `street` and `city` components in the next steps.

In [60]:
df['purchase_address'].head()

0            917 1st St, Dallas, TX 75001
2       682 Chestnut St, Boston, MA 02215
3    669 Spruce St, Los Angeles, CA 90001
4    669 Spruce St, Los Angeles, CA 90001
5       333 8th St, Los Angeles, CA 90001
Name: purchase_address, dtype: object

###  **Assigning Street and City**

After splitting the `purchase_address`, the street number was removed to isolate the street name. The `city` was directly assigned from the second part of the split:

- `street`: Extracted by removing the leading house number from the address.
- `city`: Directly taken from the second element of the split string.

This prepares the data for city-level analysis.


In [61]:
# assigning street and city
df['street'] = address_split[0].str.replace(r'^\d+\s+', '', regex=True)
df['city'] = address_split[1]


df[['street', 'city']]

,street,city
0,1st St,Dallas
2,Chestnut St,Boston
3,Spruce St,Los Angeles
4,Spruce St,Los Angeles
5,8th St,Los Angeles
...,...,...
186845,Highland St,Los Angeles
186846,Dogwood St,San Francisco
186847,12th St,San Francisco
186848,Forest St,San Francisco


### **Mapping State Abbreviations to Full Names**

To enhance readability and consistency in the dataset, a dictionary was created to map U.S. state abbreviations (e.g., `CA`, `NY`) to their full state names (e.g., `California`, `New York`). This mapping supports clearer analysis and visualization at the state level.


In [62]:
# Creating state abbreviations dictionary

us_state_abbrev = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
    'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
    'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC': 'District of Columbia'
}


### **Extracting State and ZIP Code**

The third part of the `purchase_address` string was split to isolate the **state abbreviation** and **ZIP code**. Additional columns were created:

- `state`: Extracted state abbreviation (e.g., `CA`).
- `state_name`: Mapped full state name using a predefined dictionary.
- `zip`: Extracted ZIP code for more granular geographic analysis.

In [63]:
# extracting state and zip
state_zip_split = address_split[2].str.strip().str.split(' ', expand=True)

In [64]:
# Creating State, State Name, and ZIP columns

df['state'] = state_zip_split[0]
df['state_name'] = df['state'].map(us_state_abbrev)

df['zip'] = state_zip_split[1]

### **Creating the Sales Column**

A new column `sales` was created by multiplying `quantity_ordered` and `price_each`. This represents the total revenue generated per transaction and is essential for revenue-based analysis.


In [65]:
# sales column
df['sales'] = df['quantity_ordered'] * df['price_each']

### **Extracting Date and Time Components**

The `order_date` column was decomposed into multiple time-based features for more granular temporal analysis:

- `month`: Numeric month (e.g., 1 for January)
- `month_name`: Full month name (e.g., "January")
- `year`: Year of the order
- `hour`: Hour of the day the order was placed
- `day_of_week`: Name of the day (e.g., "Monday")

These columns are useful for identifying trends and patterns across different time intervals.

In [66]:
# Separating date parts into columns

df['month'] = df['order_date'].dt.month
df['month_name'] = df['order_date'].dt.month_name()
df['year'] = df['order_date'].dt.year

df['hour'] = df['order_date'].dt.hour
df['day_of_week'] = df['order_date'].dt.day_name()

### **Extracting Additional Time Features**

Further time-based features were extracted from the `order_date` column:

- `year`: The year of each order.
- `hour`: The hour the order was placed, useful for identifying peak order times.
- `day_of_week`: The day name (e.g., Monday, Tuesday), helpful for analyzing weekly trends.

These features enhance temporal insights during exploratory data analysis.

In [67]:
df['year'] = df['order_date'].dt.year

df['hour'] = df['order_date'].dt.hour
df['day_of_week'] = df['order_date'].dt.day_name()

### **Checking for Invalid Numerical Values**

The dataset was inspected for negative or zero values in key numerical columns that should only contain positive values:

- `quantity_ordered` ≤ 0  
- `price_each` ≤ 0  
- `sales` ≤ 0

In [68]:
# Checking for negative numerical values

df[df['quantity_ordered'] <= 0]

,order_id,product,quantity_ordered,price_each,order_date,purchase_address,street,city,state,state_name,zip,sales,month,month_name,year,hour,day_of_week


In [69]:
df[df['price_each'] <= 0]

,order_id,product,quantity_ordered,price_each,order_date,purchase_address,street,city,state,state_name,zip,sales,month,month_name,year,hour,day_of_week


In [70]:
df[df['sales'] <= 0]

,order_id,product,quantity_ordered,price_each,order_date,purchase_address,street,city,state,state_name,zip,sales,month,month_name,year,hour,day_of_week


### **Sorting the Data by Order Date**

To ensure chronological consistency for time-based analysis, the dataset was sorted in ascending order based on the `order_date` column.

In [71]:
# Sort by date

df.sort_values('order_date', inplace=True)

### **Exporting the Cleaned Dataset**

The cleaned dataset was exported to a CSV file for use in further analysis and dashboard development.

In [72]:
df.to_csv('../data/cleaned_data.csv', index=False)

---

## **Wrapping Up Data Cleaning**

The data cleaning process ensured that the dataset was accurate, consistent, and suitable for analysis. The key steps included:

- Handling missing and duplicate values  
- Correcting data types for numerical and datetime fields  
- Parsing structured information from the address column  
- Removing invalid records (e.g., non-numeric values or improperly formatted rows)  
- Creating derived columns for enhanced analysis (e.g., `sales`, `month_name`, `state_name`)  

With these steps completed, the dataset is now well-structured and ready for exploratory data analysis and visualization.


---
